<div class="jumbotron m-0">
    <hr />
    <h1 class="text-center">
        <span class="text-primary">CheckPoint 3: ROS2 Basics C++ </span>
    </h1>
    <hr />
</div>

<div class="bg-primary text-center">
    - Summary -
</div>

**In this rosject, you will have to apply all that you have learned throughout the ROS2 Basics course to a real robot.**

- You will practice with a simulation and a real robot Turtlebot3
- You will launch the robot simulation here
- The real robot is running in Barcelona, Spain. You will connect remotely to it from this rosject

![sim_and_real](images/sim_and_real_lab.png)

<div class="bg-primary text-center">
    - End of Summary -
</div>

<div>
    <h1 class="text-center">
        <span class="text-primary">Section 0</span>
        &nbsp;
        <span class="">Launch the simulation</span>
    </h1>
</div>

Before working on your code, you need a simulation (or an actual robot) to be running. During the **ROS2 Basics** course, you saw the simulation running in the top right corner. The simulation was automatically launched.

In real development environments, you decide when to launch the simulation in order to start working. 

Now, let's launch the simulation we will need for this project:

1. Open a terminal by pressing on the *terminal icon* at the bottom left croner of your screen.

2. On the opened terminal, type the following ROS command that launches the simulation

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in a Terminal
</span>

In [ ]:
source /opt/ros/noetic/setup.bash
source ~/simulation_ws/devel/setup.bash
roslaunch realrobotlab main.launch

**Wait approximately 30 seconds** for the simulation to start. The simulation should automatically appear in a Gazebo window.

If it doesn't automatically appear, open yourself the Gazebo window, using the Gazebo icon:

<img src="gazebo_superapp.png" align="left" width="50"/>

<img src="new_sim.png"/>

**IMPORTANT:** the terminal that has the simulation running is occupied by it, so you cannot use it unless you kill the simulation. So you will need to open other terminals for other purposes while keeping the simulation running

<div>
    <h1 class="text-center">
        <span class="text-primary">0.1</span>
        &nbsp;
        <span class="">Launch the ros1_bridge</span>
    </h1>
</div>

The simulation of Turtlebot3 runs on ROS1, but your programs will run on ROS2. Hence, you will need to launch also the `ros1_bridge` program to allow communication between ROS1 and ROS2.

For the project we will use the `parameter_bridge` instead of the `dynamic_bridge` because the former allows to select only the topics we want to bridge. This allows important topics like `scan` to go through the bridge at a higher frequency.

<div class="bg-info text-center">
    - Notes -
</div>

If you want to learn more about the ROS1 Bridge, you can have a look at the following notebook:

[ROS1 Bridge Notebook](./extra_files/ros1bridge.ipynb)

<br>
<div class="bg-info text-center">
    - End of Notes -
</div>

### ros1_bridge launch

1. Open another terminal by clicking the *terminal icon* at the bottom left of your screen and the PLUS icon

<img src="open_terminal_superapp.png"/>

2. Then on that terminal, type the following commands to launch the bridge:

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in a Shell
</span>

In [ ]:
source ~/catkin_ws/devel/setup.bash
roslaunch load_params load_params.launch
source /opt/ros/foxy/setup.bash
ros2 run ros1_bridge parameter_bridge

You'll see the topic connections being established. From that moment, you can communicate with the topics offered by the simulation from ROS2.

<div>
    <h1 class="text-center">
        <span class="text-primary">0.2</span>
        &nbsp;
        <span class="">Set the git environment</span>
    </h1>
</div>

You need to set first a git environment to have all your code.

1. Create a new git repo in your preferred online git environment (github, gitlab, Atlassian, ...). Call it `citylab_project`. 

2. Clone this repo in the `~/ros2_ws/src` directory of this rosject.

3. Add all the packages of this project there.

**Using Git is mandatory for this project.** If you do not use Git, your assignment will not be reviewed, and you will need to redo the project using Git. Make a habit of committing your work daily or at least each time you complete one specific task. Also exercise writing clean meaningful commit messages that briefly describe the changes made in the commit. If you do not follow these instructions, it won't be possible for the team to correct some of the tasks.

<div>
    <h1 class="text-center">
        <span class="text-primary">Task 1</span>
        &nbsp;
        <span class="">Topics</span>
    </h1>
</div>

In this part of the project, you will practice using topics to control a robot. Your goal is to create a ROS program that makes the robot have a patrolling behavior.

<h1 align="center";> </h1>
<div>
    <h1 class="text-center">
        <span class="text-primary">1.1</span>
        &nbsp;
        <span class="">Patrol Behavior</span>
    </h1>
</div>

**Remember that you can check the ROS2 Basics course clicking on the notebooks icon in the bottom menu bar.**

The patrol behavior is a behavior that makes the robot **move around a specific area indefinitely while avoiding the obstacles in it**. This means that the robot must be moving, while making sure it doesn't get stuck or it hits any obstacle, the entire time.

To achieve this behavior in the robot, you need to do the following steps:

1. **Create a ROS2 package named `robot_patrol`**

    * This package is the one that will contain the full project
    * Inside the package include a ROS C++ file named `patrol.cpp`
    
    
2. **Create a C++ class**

    * Inside the `patrol.cpp` file, create a clase named **`Patrol`** that will contain the code of your node.


3. **Get the laser data**

    * Inside the `patrol.cpp` file, you need to subscribe to the laser topic and capture the rays
    

4. **Find the safest direction to move next**

    * For the sake of simplicity, you can use only the laser rays in the 180 degrees of the front (see image below).

<img src="images/laser_scope.png" width="500" />

Implement the following algorithm inside the laser callback, to identify the safest direction to move next:

* You need to get the rays corresponding to those 180º
* Identify the largest distance ray (which is not an `inf`) of those 180º, and get its corresponding angle from the front `X` axis
    * **Note**: that angle must be between $-\pi /2$ and  $\pi/2$  
* That angle indicates the direction to which the robot must rotate in order to move to the most opened (safest) area.
* Store the angle into a class variable named `direction_`

This is a VERY simple algorithm to perform a patrol. It is very simple. We will improve it later on the services section.

5. **Move the robot to the safest area**

In order to move the robot to the safe direction, you need to ublish to the velocity topic of the robot the proper values:
    
   * Inside the `patrol.cpp`, create a publisher to the `/cmd_vel` topic (the one that controls the wheels)
   * Create a control loop of 10 Hz
   * At every step of the control loop, you need to publish the proper velocity command on that topic, based on the values detected by the laser.
      * The `linear velocity in x` must always be  0.1 m/s
      * Take the value of the var `direction_` and use it to compute the proper `angular velocity in z`:
      * `angular velocity in z` = `direction_` / 2    

6. **Create a launch file named `start_patrolling.launch.py`** that starts your program.

<h1 align="center";> </h1>
<div>
    <h1 class="text-center">
        <span class="text-primary">1.2</span>
        &nbsp;
        <span class="">Test your code in the simulation</span>
    </h1>
</div>

**If the program doesn't work on the simulation, it is 100% not going to work on the real robot.**

Real-life robot development works this way: first, test your program in a simulation. When it works there, try it on a real robot. Never the other way around.

<img src="images/patrol_sim.gif"/>

Follow the steps below to test your program in the simulation:

1. Make sure the simulation is launched, as explained above
   
2. Now launch your ROS program and see the results. If the robot doesn't behave correctly, debug your program and try to figure out the reason.

<h1 align="center";> </h1>
<div>
    <h1 class="text-center">
        <span class="text-primary">1.3</span>
        &nbsp;
        <span class="">Test your code in the real robot</span>
    </h1>
</div>

Once the robot is performing as expected in the simulated environment, it is time for you to test it on the real robot.

The steps are:

1. Book a date and time in the *Real Robot Lab* to test your program (see Appendix below that teaches how to do that).


2. On the day and time selected, open this rosject


3. Connect to the real robot lab from within this rosject (see Appendix below)


4. Use the web-joystick that appears on the cameras window and move the real robot to the proper position to start before you launch your program


5. You don't need to launch the simulation, because you are using the real thing


6. **IMPORTANT**: you need to launch the `ros1_bridge` because the real Turtlebot3 also runs ROS1. Unless you launch it, your program will not be able to talk to the robot


5. Launch your program and see if it works properly. Chances are it will not because simulation is not an exact copy of the real environment. Now is your time to debug


6. Test on the real robot and debug your errors until you have it working as intended

<div class="bg-warning text-center">
    - Grading Guide -
</div>

* Check that the code is inside a git and that it has at least 5 commits at different times and days **- 0.5 points**

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in Terminal
</span>

In the directory of the student code:

In [ ]:
git log

* When you launch the `start_patrolling.launch.py` file, the robot starts moving around the city environment, while successfully avoiding the obstacles around it - **2.5 points**

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in Terminal
</span>

In [ ]:
ros2 launch robot_patrol start_patrolling.launch.py

<img src="realrobotlab_notebooks/realrobot_follow_wall.gif"/>

<div class="bg-warning text-center">
    - End Grading Guide -
</div>

<div>
    <h1 class="text-center">
        <span class="text-primary">Task 2</span>
        &nbsp;
        <span class="">Services</span>
    </h1>
</div>

In the previous section, you implemented a ver simple obstacle avoidance algorithm.

In this section, you will create a ROS Service that tells the robot what to do. This will replace the obstace avoidance algorithm you created in Task 1.

The goal in this section is to create a ROS Service that, when called:

* It receives the laser data captured by the sensor
* It analyzes the laser data received
* Based on this data, it tells the robot what to do next

To accomplish this, you will have to do 3 things:

1. Create a service server node that, when called, will analyze the laser data and tell the robot where to move next
2. Modify the program from Task 1 so that it includes now a service client that calls the service server **before** sending velocities to the robot
3. Create a new launch file to launch both nodes: the *service server* node and the *patrol* node.

**LET'S GOT FOR IT!**

<h1 align="center";> </h1>
<div>
    <h1 class="text-center">
        <span class="text-primary">2.1</span>
        &nbsp;
        <span class="">Create Service Server</span>
    </h1>
</div>

1. **Create the C++ file**

    * Inside the `robot_patrol` package create a new C++ file named `direction_service.cpp`
    
    
2. **Create the service server**

    * Inside the `direction_service.cpp` file, create a clase named **`DirectionService`** that will contain the code of your node.
    * Include on that class a service server named `/direction_service`
    * Include everything required to have a *main* that launches the server


3. **Analyze the laser data**

    * When called, the `/direction_service` will receive the current laser data from the caller
    * Then. divide the laser rays received into 3 different sections of 60º each, as described in the picture

<img src="images/laser_directions.png" width="500" />

   * Decide which is the safest direction for the robot to move next as follows:
        * Get all the distances measured for each section and add them up:
            * `total_dist_sec_right`
            * `total_dist_sec_front`
            * `total_dist_sec_left`
            
        * The section that has the largest total, is the section towards which the robot must move to

This is still a very simple algorithm for patrolling, but a little bit more robust than the one of previous section.

5. **Tell the robot what to do**

    * Depending on the laser readings analyzed, return a response
    * The service will return 1 of the following 3 responses:
        * **`forward`**: Move forward
        * **`left`**: Turn left
        * **`right`**: Turn right
        

6. **Create a custom interface**

    * The server must use a new `GetDirection` interface that you must also create, and that has the following structure:

<span class="badge badge-pill badge-primary">
    <i class="fa fa-file"></i>
    &nbsp;
    GetDirection.srv
</span>

In [ ]:
sensor_msgs/LaserScan laser_data
---
string direction

8. **Launch the server**

    * Create a launch file named `start_direction_service.launch.py` that launches the `direction_service.cpp` program.

### Test the Service Server

How can you test and debug the service server independently of the code of Task 1? By doing the following steps:

1. Inside the `robot_patrol` package create a new C++ file named `test_service.cpp`


2. Inside this file, create a simple node that does the following:
    * It subscribes to the laser data
    * In the callback, it calls the `/direction_service` service with the proper data
    * It prints the response returned by the service
    

3. Create a launch file named `start_test_service.launch.py` that launches the `test_service.cpp` program.    


4. Put the simulated robot in a proper location for your test. You can do that using the `teleop_twist_keyboard` node (see Notes below)


5. Launch the service server node using the `start_direction_service.launch.py` file


6. Launch the tester node using the `start_test_service.launch.py` file

<div class="bg-info text-center">
    - Notes -
</div>

To move the simulated robot using the `teleop_twist_keyboard` node, run the following command:

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in Shell
</span>

In [ ]:
rosrun teleop_twist_keyboard teleop_twist_keyboard.py

<div class="bg-info text-center">
    - End of Notes -
</div>

<h1 align="center";> </h1>
<div>
    <h1 class="text-center">
        <span class="text-primary">2.2</span>
        &nbsp;
        <span class="">Add service call to Task 1 program</span>
    </h1>
</div>

Once the service is working, you need to call it from the `patrol.cpp` program that you created in Task 1. 

The idea is that the main node will continuously call the `/direction_service` in order to decide where the robot will move next.

So you need to do the following:

1. Create a duplicate of the `patrol.cpp` file, named `patrol_with_service.cpp`
2. Add to the `patrol_with_service.cpp` node a service client that connects to the `/direction_service` service
3. Use this client to call the service every time before deciding the next robot movement (inside the control loop). 

    For that you need:
    * Remove the computation of the `direction_` in the laser callback.
    * Now, in the laser callback, just store the latest laser message ina class variable named `last_laser_`
    * In the control loop, call the service providing the `last_laser_` data
    * Based on the response obtained from the service, assign the following speeds to the robot:
        1. If `front` then `linear.x=0.1` and `angular.z=0.0`
        2. If `left` then `linear.x=0.1` and `angular.z=0.5`
        3. If `right` then `linear.x=0.1` and `angular.z=-0.5`


<h1 align="center";> </h1>
<div>
    <h1 class="text-center">
        <span class="text-primary">2.3</span>
        &nbsp;
        <span class="">Create new launch file</span>
    </h1>
</div>

Now you need to create a launch file that launched both the `patrol_with_service.cpp` and the `direction_service.cpp` programs, because both of them have to be running in order to work together.

1. Create a new launch file named `main.launch.py` that launches both nodes: first the service server node and then the patrolling node

2. Run the `main.launch.py` file to start the full program

This should start the robot patrolling behavior.

<h1 align="center";> </h1>
<div>
    <h1 class="text-center">
        <span class="text-primary"></span>
        &nbsp;
        <span class="">Test your code in the real robot</span>
    </h1>
</div>

Once the robot is performing as expected in the simulated environment, it is time for you to test it on the real robot.

The steps are:

1. Book a date and time in the *Real Robot Lab* to test your program (see Appendix below that teaches how to do that).


2. On the day and time selected, open this rosject


3. Connect to the real robot lab from within this rosject (see Appendix below)


4. Use the web-joystick that appears on the cameras window and move the real robot to the proper position to start before you launch your program


5. You don't need to launch the simulation, because you are using the real thing


6. **IMPORTANT**: you need to launch the `ros1_bridge` because the real Turtlebot3 also runs ROS1. Unless you launch it, your program will not be able to talk to the robot


5. Launch your program and see if it works properly. Chances are it will not because simulation is not an exact copy of the real environment. Now is your time to debug


6. Test on the real robot and debug your errors until you have it working as intended

<div class="bg-warning text-center">
    - Grading Guide -
</div>

* When you launch the `start_direction_service.launch.py` and `start_test_service.launch.py` files, the service works as expected - **2 points**

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in Terminal
</span>

In [ ]:
ros2 launch robot_patrol start_direction_service.launch.py

In [ ]:
ros2 launch robot_patrol start_test_service.launch.py

<img src="images/test_service.gif" width="800" />

* When you launch the `main.launch.py` file, the robot starts moving around the city environment, while successfully avoiding the obstacles around it (using the service `/direction_service`) - **1.5 points**

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in Terminal
</span>

In [ ]:
ros2 launch robot_patrol main.launch.py

<div class="bg-warning text-center">
    - End Grading Guide -
</div>

<div>
    <h1 class="text-center">
        <span class="text-primary">Task 3</span>
        &nbsp;
        <span class="">Actions</span>
    </h1>
</div>

### Create an action server to send position goals to the robot

The goal of this section is to **create an action server that allows you to send the robot to a certain position**, so we can later use it to manually navigate the the robot around the environment.

To achieve that, you will need to create another node that contains an action server in charge of doing the necessary operations.

<img src="images/t3_pos_goal.png" width="500" />

- `x`: x coordinate (in meters)
- `y`: y coordinate (in meters)
- `theta`: goal orientation (in degrees)

<h1 align="center";> </h1>
<div>
    <h1 class="text-center">
        <span class="text-primary">3.1</span>
        &nbsp;
        <span class="">Create action server</span>
    </h1>
</div>

To achieve that, make the following steps:

1. **Create the C++ file**

    * Inside the `robot_patrol` package create a new C++ file named `go_to_point_action.cpp`
    
    
2. **Create a C++ class**

    * Inside the `go_to_point_action.cpp` file, create a clase named **`GoToPoint`** that will contain the code of your node.


3. **Create the action server**

    * Inside the `go_to_point_action.cpp` file, create an action server named `/go_to_point`
    * In the callback, store the desired position received into a class variable named:
        * `Point32 desired_pos_`
    

4. **Get the odometry data**

    * Inside the `go_to_point_action.cpp` file, you need to subscribe to the odometry topic `/odom`
    * In the callback of `/odom`, store the current odometry `x`, `y` and `theta` into the class var:
        * `Point32 current_pos_`

    * **Important**: remember that the odometry is received in quaternion format. To get the `theta` value you will need to convert the quaternion into Euler angles.
    

5. **Publish to the velocity topic of the robot**
    
    * Also inside the `go_to_point_action.cpp`, create a publisher to the `/cmd_vel` topic that controls the wheels
    * At every step of the control loop, you need to publish the proper velocity command on that topic, based on the odometry readings.
    
    
6. **Create the control loop**
    
    * In order to reach the desired goal, you will need to implement a basic control loop:
    
    1. Get the current position `current_pos_`
    2. Get the desired position `desired_pos`
    3. Compute the difference between the two.
    4. This difference defines a vector with the direction you need to move
    5. Convert the vector direction into the angular speed (`angular.z`) required, using the same principle as in section 1
    6. Send the speed to the `/cmd_vel` applying a fixed `linear.x` of 0.2 m/s


7. **Create a custom interface**

    * The server must use a new `GoToPoint.action` interface that you must also create, and that has the following structure:

<span class="badge badge-pill badge-primary">
    <i class="fa fa-file"></i>
    &nbsp;
    GoToPoint.action
</span>

In [ ]:
geometry_msgs/Point32 goal_pos
---
bool status
---
geometry_msgs/Point32 current_pos

8. **Create the launch file**

    * Create a launch file named `start_gotopoint_action.launch.py` that launches the `go_to_point_action.cpp` program.

### Action Behavior

* The action server will receive a `Point32` message as a goal. This `Point32` will define the goal position [x, y, theta] for the robot.

* As feedback, the action server publishes the current position of the robot [x,y,theta] every second.

* When the action finishes, the result will return the following:

    * A boolean with True inndicating the action finished correctly

<span style="color:red;">**IMPORTANT:** For reaching the goal, it is not necessary to perform obstacle avoidance. Therefore, you can assume there are no obstacles in the environment.</span>

<h1 align="center";> </h1>
<div>
    <h1 class="text-center">
        <span class="text-primary">3.2</span>
        &nbsp;
        <span class="">Test your code in the real robot</span>
    </h1>
</div>

Once the robot is performing as expected in the simulated environment, it is time for you to test it on the real robot.

The steps are:

1. Book a date and time in the *Real Robot Lab* to test your program (see Appendix below that teaches how to do that).


2. On the day and time selected, open this rosject


3. Connect to the real robot lab from within this rosject (see Appendix below)


4. Use the web-joystick that appears on the cameras window and move the real robot to the proper position to start before you launch your program


5. You don't need to launch the simulation, because you are using the real thing


6. **IMPORTANT**: you need to launch the `ros1_bridge` because the real Turtlebot3 also runs ROS1. Unless you launch it, your program will not be able to talk to the robot


5. Launch your program and see if it works properly. Chances are it will not because simulation is not an exact copy of the real environment. Now is your time to debug


6. Test on the real robot and debug your errors until you have it working as intended

<div class="bg-warning text-center">
    - Grading Guide -
</div>

* The custom action interface has been created successfully - **0.5 points**

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in Terminal
</span>

In [ ]:
ros2 interface show robot_patrol/action/GoToPoint

<pre style="color: white; background: black; padding:10px; font: 1.3rem Inconsolata, monospace;">
geometry_msgs/Point32 goal_pos
---
bool status
---
geometry_msgs/Point32 current_pos
</pre>

* When you launch the `start_gotopoint_action.launch.py` file, the action server is started and you can send position goals to it. The robot correctly reaches the goal position - **2.5 points**

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in Terminal
</span>

In [ ]:
ros2 launch robot_patrol start_gotopoint_action.launch.py

In [ ]:
ros2 action send_goal -f /go_to_point robot_patrol/action/GoToPoint "goal_pos:
  x: 0.7
  y: 0.3
  z: 0.0
"

<img src="images/go_to_point_action.gif" width="800" />

* The action server is able to handle properly consecutive goals - **0.5 points**

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in Terminal
</span>

In [ ]:
ros2 launch robot_patrol start_gotopoint_action.launch.py

In [ ]:
ros2 action send_goal -f /go_to_point robot_patrol/action/GoToPoint "goal_pos:
  x: 0.7
  y: 0.3
  z: 0.0
"

In [ ]:
ros2 action send_goal -f /go_to_point robot_patrol/action/GoToPoint "goal_pos:
  x: 0.7
  y: 0.0
  z: 90.0
"

<div class="bg-warning text-center">
    - End Grading Guide -
</div>

<div>
    <h1 class="text-center">
        <span class="text-primary">Appendix</span>
        &nbsp;
        <span class="">How to connect to the real robot</span>
    </h1>
</div>

Once you know the basics of the operation with a simulated version, it's time to practice what you've learned with a TurtleBot3 robot.

### Book up to two 25-minute sessions per week

To make a booking, follow these simple steps:

<div>
    <h1 class="text-center">
        <span class="text-primary">Step 1</span>
        &nbsp;
        <span class="">Book a Session</span>
    </h1>
</div>

From The Construct's main dashboard, book a session by clicking on the icon seen below:

<img src="realrobotlab_notebooks/reallab2.png"/>

A booking page will appear, where you can make a new booking.

<img src="realrobotlab_notebooks/bookings.png"/>

You can either click on `Book a robot` on the bottom and then select the Turtlebot, or just click on `Book now` where the image of the turtlebot is and select the date and time for your reservation:

<img src="realrobotlab_notebooks/reservet3.png"/>

Only the **available dates and times** are shown in green. They come in **25-minute blocks**.

There is also a limitation on the number of bookings per week a user can make.

Your number of available bookings depends on your license and subscription.

<img src="realrobotlab_notebooks/confirm.png"/>

* At any point in time you can check for bookings you have:

<img src="realrobotlab_notebooks/bookasession.png"/>

<div>
    <h1 class="text-center">
        <span class="text-primary">Step 2</span>
        &nbsp;
        <span class="">Open project rosject</span>
    </h1>
</div>

Before the time of your booking, open your rosject. You will see the robot icon at the bottom bar has an orange dot. It means that your time to connect to the roobt will start soon. If you click on it you will see a countdown to your session start.

<img src="realrobotlab_notebooks/remaining.png"/>

The orange dot will turn red when your booked session has started.

<div>
    <h1 class="text-center">
        <span class="text-primary">Step 3</span>
        &nbsp;
        <span class="">Turn ON robot connection</span>
    </h1>
</div>

Once you are inside the rosject and your real robot session has started, click on the robot icon at the bottom. A popup with a toggle for **turning on the connection** will appear.

**WARNING: Nothing will appear unless you have a booking. So if you didn't make a booking in the dashboard, now is the time to do it.**

Now you just have to click on the toggle to connect the desktop environment to the remote lab. This will start the connection process, which lasts for a few seconds. 

Once the connection process is finished, the **red dot will change to green** if succesfully connected. Now **any terminal you open will show the topics and services of the real robot**.

Move the joystick around to confirm you are connected to the robot:

<img src="realrobotlab_notebooks/rrl.gif"/>

You can perform the following command, generating a list of topics similar to the list below. The first time you issue a ROS command, it could take an extra 30 seconds to appear:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
rostopic list

In [ ]:
/battery_state
/cmd_vel
/cmd_vel_rc100
/diagnostics
/firmware_version
/imu
/joint_states
/magnetic_field
/motor_power
/odom
/reset
/rosout
/rosout_agg
/rpms
/scan
/sensor_state
/sound
/tf
/tf_static
/version_info

## Ready for your programs!

If you have reached this point, it means that you have a proper connection with the remote real robot lab. **Now you can launch the programs of your project and see the results on a real robot**.